In [32]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from math import pi
from datetime import datetime  as dt
import matplotlib
import networkx as nx

# Import API key
from api_keys import api_key
single_city_url = "http://api.openweathermap.org/data/2.5/weather?"

In [33]:
#import bokeh for analysis of datasets.
from bokeh.models import DatetimeTickFormatter
from bokeh.plotting import figure, output_notebook, show, ColumnDataSource
from bokeh.models.tools import HoverTool
import bokeh.palettes as bcolors
from bokeh.transform import factor_cmap
import matplotlib.dates as mdates

In [34]:
output_notebook()

Loading BokehJS ...

In [19]:
city_name = "portland"
city_id = "5746545"
city_units = "metric"
covid = "mobility-trends.csv"

In [20]:
covid_df = pd.read_csv(covid)

In [21]:
covid_df

,geo_type,region,transportation_type,date,value
0,country/region,Albania,driving,2020-01-13,100.00
1,country/region,Albania,driving,2020-01-14,95.30
2,country/region,Albania,driving,2020-01-15,101.43
3,country/region,Albania,driving,2020-01-16,97.20
4,country/region,Albania,driving,2020-01-17,103.55
...,...,...,...,...,...
41470,city,Zurich,walking,2020-04-22,72.35
41471,city,Zurich,walking,2020-04-23,73.64
41472,city,Zurich,walking,2020-04-24,76.72
41473,city,Zurich,walking,2020-04-25,78.56


In [22]:
covcopy = covid_df.copy()

In [23]:
region_type_list = covid_df['geo_type'].unique()

In [44]:
move_type_list = covid_df['transportation_type'].unique()

In [45]:
move_type_list

array(['driving', 'walking', 'transit'], dtype=object)

In [24]:
region_list_city = covid_df.loc[covid_df['geo_type'] == "city"]['region'].unique()

In [25]:
region_list_city

array(['Amsterdam', 'Athens', 'Atlanta', 'Auckland', 'Baltimore',
       'Bangkok', 'Barcelona', 'Berlin', 'Birmingham - UK',
       'Bochum - Dortmund', 'Boston', 'Brisbane', 'Brussels',
       'Buenos Aires', 'Cairo', 'Calgary', 'Cape Town', 'Chicago',
       'Cologne', 'Copenhagen', 'Dallas', 'Delhi', 'Denver', 'Detroit',
       'Dubai', 'Dublin', 'Dusseldorf', 'Edmonton', 'Frankfurt',
       'Fukuoka', 'Guadalajara', 'Halifax', 'Hamburg', 'Helsinki',
       'Houston', 'Hsin-chu', 'Istanbul', 'Jakarta', 'Johannesburg',
       'Kuala Lumpur', 'Leeds', 'Lille', 'London', 'Los Angeles', 'Lyon',
       'Madrid', 'Manchester', 'Manila', 'Melbourne', 'Mexico City',
       'Miami', 'Milan', 'Montreal', 'Moscow', 'Mumbai', 'Munich',
       'Nagoya', 'New York City', 'Osaka', 'Oslo', 'Ottawa', 'Paris',
       'Perth', 'Philadelphia', 'Rio de Janeiro', 'Riyadh', 'Rome',
       'Rotterdam', 'Saint Petersburg', 'San Francisco - Bay Area',
       'Santiago', 'Sao Paulo', 'Seattle', 'Seoul', 'Sto

In [26]:
region_list_country = covid_df.loc[covid_df['geo_type'] == "country/region"]['region'].unique()

In [27]:
region_list_country

array(['Albania', 'Argentina', 'Australia', 'Austria', 'Belgium',
       'Brazil', 'Bulgaria', 'Cambodia', 'Canada', 'Chile', 'Colombia',
       'Croatia', 'Czech Republic', 'Denmark', 'Egypt', 'Estonia',
       'Finland', 'France', 'Germany', 'Greece', 'Hong Kong', 'Hungary',
       'Iceland', 'India', 'Indonesia', 'Ireland', 'Israel', 'Italy',
       'Japan', 'Latvia', 'Lithuania', 'Luxembourg', 'Macao', 'Malaysia',
       'Mexico', 'Morocco', 'Netherlands', 'New Zealand', 'Norway',
       'Philippines', 'Poland', 'Portugal', 'Republic of Korea',
       'Romania', 'Russia', 'Saudi Arabia', 'Serbia', 'Singapore',
       'Slovakia', 'Slovenia', 'South Africa', 'Spain', 'Sweden',
       'Switzerland', 'Taiwan', 'Thailand', 'Turkey', 'UK',
       'United States', 'Ukraine', 'United Arab Emirates', 'Uruguay',
       'Vietnam'], dtype=object)

In [28]:
city_df =  covid_df.loc[covid_df['geo_type'] == "city"]
city_df

,geo_type,region,transportation_type,date,value
16065,city,Amsterdam,driving,2020-01-13,100.00
16066,city,Amsterdam,driving,2020-01-14,103.53
16067,city,Amsterdam,driving,2020-01-15,104.23
16068,city,Amsterdam,driving,2020-01-16,103.92
16069,city,Amsterdam,driving,2020-01-17,111.95
...,...,...,...,...,...
41470,city,Zurich,walking,2020-04-22,72.35
41471,city,Zurich,walking,2020-04-23,73.64
41472,city,Zurich,walking,2020-04-24,76.72
41473,city,Zurich,walking,2020-04-25,78.56


In [29]:
zurich = city_df.loc[city_df['region'] == "Zurich"]
zurich

,geo_type,region,transportation_type,date,value
41160,city,Zurich,driving,2020-01-13,100.00
41161,city,Zurich,driving,2020-01-14,102.38
41162,city,Zurich,driving,2020-01-15,110.84
41163,city,Zurich,driving,2020-01-16,105.48
41164,city,Zurich,driving,2020-01-17,113.83
...,...,...,...,...,...
41470,city,Zurich,walking,2020-04-22,72.35
41471,city,Zurich,walking,2020-04-23,73.64
41472,city,Zurich,walking,2020-04-24,76.72
41473,city,Zurich,walking,2020-04-25,78.56


In [43]:
zurich.describe(include='all')

,geo_type,region,transportation_type,date,value
count,315,315,315,315,315.000000
unique,1,1,3,105,NaN
top,city,Zurich,walking,2020-02-03,NaN
freq,315,315,105,3,NaN
mean,NaN,NaN,NaN,NaN,82.472286
std,NaN,NaN,NaN,NaN,28.385619
min,NaN,NaN,NaN,NaN,24.230000
25%,NaN,NaN,NaN,NaN,62.200000
50%,NaN,NaN,NaN,NaN,92.130000
75%,NaN,NaN,NaN,NaN,104.530000


In [30]:

ref_date = []

for date in zurich['date']:
    ref_date.append(np.datetime64(dt.strptime(date, "%Y-%m-%d")))

In [36]:
date_cvd = []

for date in zurich['date']:
    date_cvd.append(str(date))
    #halfdate = str(date).split(',')    #decided to keep time stamp, 
    #date_str_cme.append(halfdate[0])

# pass plot data as well as hovertool data in to plot.
source_cvd = ColumnDataSource(
        data=dict(
            x=ref_date,
            y=zurich['value'],
            desc=date_cvd,
        )
    )

In [35]:
grp_color = []
grp_color_palette = bcolors.brewer['Spectral'][6]
#linked_evt_count
for flr in range(6):
    grp_color.append(grp_color_palette[flr])
    

In [37]:
# configure plot tools including the hover tool / tooltips.  Displays date / time of individual data points when hovering 
#over individual points.
TOOLSCCD = [HoverTool(tooltips=[
            ("cvd_date", "@desc"),("cvd_value","$y")],
                  formatters={'DateTime': 'datetime'}),'wheel_zoom','save','crosshair','pan','reset']

In [40]:
cid = figure(title = f"Values for city.",plot_width=800, plot_height=600,tools=TOOLSCCD,x_axis_type="datetime") #, tooltips=TOOLTIPS
cid.xaxis.axis_label = 'Event Date'
cid.yaxis.axis_label = 'COVID Value'
# Setup x axis for using Datetime formatting.
cid.xaxis.formatter=DatetimeTickFormatter(
        hours=["%d %B %Y"],
        days=["%d %B %Y"],
        months=["%d %B %Y"],
        years=["%d %B %Y"],
    )

cid.xaxis.major_label_orientation = pi/4
cid.circle('x', 'y', line_color="navy", fill_color="red", fill_alpha=0.2, size=10,source=source_cvd)


GlyphRenderer(id='1210', ...)

In [41]:
show(cid)

thedate.strftime("%Y-%m-%d") = zurich['date'] 

In [16]:
zurich.plot(x ='date', y='value', kind = 'scatter')

ValueError: scatter requires x column to be numeric

In [6]:
covcopy.groupby(['region'])

In [7]:
covcopy

,geo_type,region,transportation_type,date,value
0,country/region,Albania,driving,2020-01-13,100.00
1,country/region,Albania,driving,2020-01-14,95.30
2,country/region,Albania,driving,2020-01-15,101.43
3,country/region,Albania,driving,2020-01-16,97.20
4,country/region,Albania,driving,2020-01-17,103.55
...,...,...,...,...,...
41470,city,Zurich,walking,2020-04-22,72.35
41471,city,Zurich,walking,2020-04-23,73.64
41472,city,Zurich,walking,2020-04-24,76.72
41473,city,Zurich,walking,2020-04-25,78.56


query_string = single_city_url + "appid=" + api_key + "&q=" + city_name

request = requests.get(query_string).json()

request

In [22]:
query_string2 = single_city_url + "appid=" + api_key + "&id=" + city_id + "&units=" + city_units

In [23]:
request2 = requests.get(query_string2).json()

In [24]:
request2

{'coord': {'lon': -122.68, 'lat': 45.52},
 'weather': [{'id': 803,
   'main': 'Clouds',
   'description': 'broken clouds',
   'icon': '04d'}],
 'base': 'stations',
 'main': {'temp': 19.18,
  'feels_like': 14.65,
  'temp_min': 17.22,
  'temp_max': 20.56,
  'pressure': 1017,
  'humidity': 32},
 'visibility': 16093,
 'wind': {'speed': 4.1, 'deg': 190},
 'clouds': {'all': 75},
 'dt': 1587942301,
 'sys': {'type': 1,
  'id': 5321,
  'country': 'US',
  'sunrise': 1587906323,
  'sunset': 1587957068},
 'timezone': -25200,
 'id': 5746545,
 'name': 'Portland',
 'cod': 200}

In [25]:
city_path = "city_list.csv"
cities_df = pd.read_csv(city_path)
city_data_dict = []
city_data_dict.append({'city_id': city_id, 'city_name' : city_name, 'country' : request2['sys']['country'], 
                       'date' : request2['dt'], 'lon' : request2['coord']['lon'], 'lat':request2['coord']['lat'], 
                       'main_temp' : request2['main']['temp'], 'min_temp' : request2['main']['temp_min'],
                       'max_temp' : request2['main']['temp_max'], 'main_pressure' : request2['main']['pressure'], 
                       'humidity' : request2['main']['humidity'], 'wind_speed' : request2['wind']['speed'], 
                       'wind_deg' : request2['wind']['deg'], 'clouds' : request2['clouds']['all'],
                      'visibility' : request2['visibility']})

In [26]:
city_data_dict

[{'city_id': '5746545',
  'city_name': 'portland',
  'country': 'US',
  'date': 1587942301,
  'lon': -122.68,
  'lat': 45.52,
  'main_temp': 19.18,
  'main_pressure': 1017,
  'humidity': 32,
  'wind_speed': 4.1,
  'wind_deg': 190,
  'clouds': 75,
  'visibility': 16093}]

In [5]:
city_path = "city_list.csv"
cities_df = pd.read_csv(city_path)

In [11]:
city_master = []
for i,row in cities_df.iterrows():
    city_master.append({"city_id" : row['city_id'],"city_name" : row['city_name']})
    #print(row['city_id']
    
for cityinfo in city_master:
    query_string = single_city_url + "appid=" + api_key + "&id=" + str(cityinfo["city_id"]) + "&units=" + city_units
    request2 = requests.get(query_string).json()
    city_data_dict = []
    city_data_dict.append({'city_id': cityinfo["city_id"], 'city_name' : cityinfo["city_name"], 'country' : request2['sys']['country'], 
                       'date' : request2['dt'], 'lon' : request2['coord']['lon'], 'lat':request2['coord']['lat'], 
                       'main_temp' : request2['main']['temp'], 'min_temp' : request2['main']['temp_min'],
                       'max_temp' : request2['main']['temp_max'], 'main_pressure' : request2['main']['pressure'], 
                       'humidity' : request2['main']['humidity'], 'wind_speed' : request2['wind']['speed'], 
                       'clouds' : request2['clouds']['all'], 'visibility' : request2['visibility']})



In [10]:
request2

{'coord': {'lon': -122.76, 'lat': 45.38},
 'weather': [{'id': 500,
   'main': 'Rain',
   'description': 'light rain',
   'icon': '10d'}],
 'base': 'stations',
 'main': {'temp': 18.38,
  'feels_like': 17.22,
  'temp_min': 16.67,
  'temp_max': 20,
  'pressure': 1016,
  'humidity': 72},
 'visibility': 16093,
 'wind': {'speed': 3.1},
 'rain': {'1h': 0.51},
 'clouds': {'all': 75},
 'dt': 1588207447,
 'sys': {'type': 1,
  'id': 6111,
  'country': 'US',
  'sunrise': 1588165282,
  'sunset': 1588216500},
 'timezone': -25200,
 'id': 5757506,
 'name': 'Tualatin',
 'cod': 200}